# Klang Strudel Playground

This notebook demonstrates playing Strudel patterns with source location tracking.

In [1]:
USE {
    repositories {
        mavenCentral()
        mavenLocal()
    }

    dependencies {
        // Load the klang-notebook module (which includes all Klang modules)
        implementation("/opt/dev/peekandpoke/klang/klang-notebook/build/libs/klang-notebook-jvm-0.1.0.jar")

        // Also load the dependent modules (since we're not using Maven)
        implementation("/opt/dev/peekandpoke/klang/tones/build/libs/tones-jvm-0.1.0.jar")
        implementation("/opt/dev/peekandpoke/klang/audio_be/build/libs/audio_be-jvm-0.1.0.jar")
        implementation("/opt/dev/peekandpoke/klang/audio_fe/build/libs/audio_fe-jvm-0.1.0.jar")
        implementation("/opt/dev/peekandpoke/klang/audio_bridge/build/libs/audio_bridge-jvm-0.1.0.jar")
        implementation("/opt/dev/peekandpoke/klang/klang/build/libs/klang-jvm-0.1.0.jar")
        implementation("/opt/dev/peekandpoke/klang/klangscript/build/libs/klangscript-jvm-0.1.0.jar")
        implementation("/opt/dev/peekandpoke/klang/strudel/build/libs/strudel-jvm-0.1.0.jar")
    }
}

%use coroutines
%use serialization
%use ktor-client

println("✅ Dependencies loaded!")

✅ Dependencies loaded!


In [2]:
// Import necessary classes
import io.peekandpoke.klang.audio_engine.KlangPlayer
import io.peekandpoke.klang.audio_engine.klangPlayer
import io.peekandpoke.klang.audio_fe.create
import io.peekandpoke.klang.audio_fe.samples.SampleCatalogue
import io.peekandpoke.klang.audio_fe.samples.Samples
import io.peekandpoke.klang.strudel.StrudelPattern
import kotlinx.coroutines.*
import java.util.concurrent.Executors

// Load samples
val samples = runBlocking { Samples.create(catalogue = SampleCatalogue.default) }
println("✅ Sample catalogue loaded successfully!")

// Create a dedicated thread pool for audio
val audioThreadPool = Executors.newFixedThreadPool(4)
val audioDispatcher = audioThreadPool.asCoroutineDispatcher()
println("✅ Audio thread pool created!")

// Create a long-lived coroutine scope for the player
val playerScope = CoroutineScope(SupervisorJob() + audioDispatcher)
println("✅ Player scope created!")

// Create the player with explicit scope and dispatchers
val player = KlangPlayer(
    options = KlangPlayer.Options(
        samples = samples,
        sampleRate = 48000,
        blockSize = 512
    ),
    backendFactory = { config ->
        io.peekandpoke.klang.audio_be.JvmAudioBackend(config)
    },
    scope = playerScope,
    fetcherDispatcher = audioDispatcher,
    backendDispatcher = audioDispatcher,
    callbackDispatcher = Dispatchers.Default,
)

println("✅ KlangPlayer created with dedicated thread pool!")
println("Player is now running in background threads")

Loading sample source: Strudel Default Drums
Loaded sample source: Strudel Default Drums ... Found 1 banks, 16 sounds, 0 aliases
Loading sample source: Tidal Drum Machine
Loaded sample source: Tidal Drum Machine ... Found 71 banks, 683 sounds, 67 aliases
Loading sample source: Dough Samples
Loaded sample source: Dough Samples ... Found 1 banks, 9 sounds, 0 aliases
Loading sample source: Vcsl Samples
Loaded sample source: Vcsl Samples ... Found 29 banks, 128 sounds, 0 aliases
Loading sample source: mridangam
Loaded sample source: mridangam ... Found 1 banks, 13 sounds, 0 aliases
Loading sample source: Piano
Loaded sample source: Piano ... Found 1 banks, 1 sounds, 0 aliases
Loading sample source: GM - Felix Roos
Loaded sample source: GM - Felix Roos ... Found 1 banks, 125 sounds, 0 aliases
✅ Sample catalogue loaded successfully!
✅ Audio thread pool created!
✅ Player scope created!
✅ KlangPlayer created with dedicated thread pool!
Player is now running in background threads


In [7]:
import io.peekandpoke.klang.strudel.lang.fast
import io.peekandpoke.klang.strudel.lang.gain
import io.peekandpoke.klang.strudel.lang.lpf
import io.peekandpoke.klang.strudel.lang.note
import io.peekandpoke.klang.strudel.lang.sound

// Test: Execute a simple Strudel pattern
//val song = sound("bd hh sd oh").fast(2).gain(1.0)
val song = sound("bd hh sd oh").fast(2).gain(1.0)
song

io.peekandpoke.klang.strudel.pattern.ControlPattern@79378ff4

In [4]:
// Extract the pattern and query events
import io.peekandpoke.klang.script.runtime.NativeObjectValue

val events = song.queryArc(0.0, 1.0)

println("Found ${events.size} events in first cycle:")
events.forEach { event ->
    val loc = event.sourceLocations?.outermost
    println("  - Event at ${loc?.source}:${loc?.startLine}:${loc?.startColumn}-${loc?.endColumn}")
    println("    Data: ${event.data}")
}

events

Found 4 events in first cycle:
  - Event at null:null:null-null
    Data: VoiceData(note=A, freqHz=220.0, scale=null, gain=1.0, legato=null, bank=null, sound=null, soundIndex=null, density=null, panSpread=null, freqSpread=null, voices=null, filters=FilterDefs(filters=[]), adsr=AdsrEnvelope(attack=null, decay=null, sustain=null, release=null), accelerate=null, vibrato=null, vibratoMod=null, distort=null, coarse=null, crush=null, cutoff=null, hcutoff=null, bandf=null, resonance=null, orbit=null, pan=null, delay=null, delayTime=null, delayFeedback=null, room=null, roomSize=null, begin=null, end=null, speed=null, loop=null, cut=null, value=null)
  - Event at null:null:null-null
    Data: VoiceData(note=B, freqHz=246.94165062806206, scale=null, gain=1.0, legato=null, bank=null, sound=null, soundIndex=null, density=null, panSpread=null, freqSpread=null, voices=null, filters=FilterDefs(filters=[]), adsr=AdsrEnvelope(attack=null, decay=null, sustain=null, release=null), accelerate=null, vibrat

[StrudelPatternEvent(begin=0.0, end=0.25, dur=0.25, data=VoiceData(note=A, freqHz=220.0, scale=null, gain=1.0, legato=null, bank=null, sound=null, soundIndex=null, density=null, panSpread=null, freqSpread=null, voices=null, filters=FilterDefs(filters=[]), adsr=AdsrEnvelope(attack=null, decay=null, sustain=null, release=null), accelerate=null, vibrato=null, vibratoMod=null, distort=null, coarse=null, crush=null, cutoff=null, hcutoff=null, bandf=null, resonance=null, orbit=null, pan=null, delay=null, delayTime=null, delayFeedback=null, room=null, roomSize=null, begin=null, end=null, speed=null, loop=null, cut=null, value=null), sourceLocations=null), StrudelPatternEvent(begin=0.25, end=0.5, dur=0.25, data=VoiceData(note=B, freqHz=246.94165062806206, scale=null, gain=1.0, legato=null, bank=null, sound=null, soundIndex=null, density=null, panSpread=null, freqSpread=null, voices=null, filters=FilterDefs(filters=[]), adsr=AdsrEnvelope(attack=null, decay=null, sustain=null, release=null), acc

In [5]:
// Simple audio test - play a 440Hz sine wave for 1 second
import javax.sound.sampled.*
import kotlin.math.*

fun testAudioOutput() {
    try {
        val sampleRate = 44100f
        val frequency = 440f  // A4 note
        val durationMs = 1000

        // Create audio format (16-bit stereo)
        val format = AudioFormat(sampleRate, 16, 2, true, false)

        // Get audio line
        val line = AudioSystem.getSourceDataLine(format)
        line.open(format)
        line.start()

        println("🔊 Playing 440Hz sine wave for 1 second...")

        // Generate samples
        val samples = (sampleRate * durationMs / 1000).toInt()
        val buffer = ByteArray(samples * 4) // 4 bytes per frame (2 channels * 16-bit)

        var idx = 0
        for (i in 0 until samples) {
            // Generate sine wave sample
            val sample = (sin(2.0 * PI * frequency * i / sampleRate) * 32767 * 0.3).toInt().toShort()

            // Write to both channels (stereo)
            // Left channel
            buffer[idx++] = (sample.toInt() and 0xFF).toByte()
            buffer[idx++] = ((sample.toInt() shr 8) and 0xFF).toByte()
            // Right channel
            buffer[idx++] = (sample.toInt() and 0xFF).toByte()
            buffer[idx++] = ((sample.toInt() shr 8) and 0xFF).toByte()
        }

        // Write to audio line
        line.write(buffer, 0, buffer.size)

        // Wait for playback to complete
        line.drain()
        line.stop()
        line.close()

        println("✅ Audio playback completed successfully!")
        println("If you heard a tone, audio output is working!")

    } catch (e: Exception) {
        println("❌ Audio playback failed: ${e.message}")
        e.printStackTrace()
    }
}

testAudioOutput()

🔊 Playing 440Hz sine wave for 1 second...
✅ Audio playback completed successfully!
If you heard a tone, audio output is working!


In [8]:
// Play the pattern (after samples are loaded!)
import io.peekandpoke.klang.strudel.playStrudel
import kotlinx.coroutines.*

// Create a playback for the pattern
val playback = player.playStrudel(song)

val deferred = runBlocking {
    println("🎵 Starting playback...")

    playback.start()

    println("Playing for 8 seconds (allowing samples to load)...")
    for (i in 1..8) {
        delay(1000)
        println("  ${i}s - Active playbacks: ${player.activePlaybacks.size}")
    }

    playback.stop()
    println("✅ Stopped")
}


🎵 Starting playback...
Playing for 8 seconds (allowing samples to load)...
[Song: playback-1] Sample Look ahead 0.0 - 8.0
[Song: playback-1] Sample Look ahead 8.0 - 9.0
Loading sample https://raw.githubusercontent.com/tidalcycles/uzu-drumkit/main/bd/10_bd_switchangel.wav
Loading sample https://raw.githubusercontent.com/tidalcycles/uzu-drumkit/main/oh/10_oh_switchangel.wav
Loading sample https://raw.githubusercontent.com/tidalcycles/uzu-drumkit/main/sd/10_sd_switchangel-bounce-2.wav
Loading sample https://raw.githubusercontent.com/tidalcycles/uzu-drumkit/main/hh/10_hh_switchangel.wav
[Song: playback-1] Sample Look ahead 9.0 - 10.0
  1s - Active playbacks: 1
[Song: playback-1] Sample Look ahead 10.0 - 11.0
  2s - Active playbacks: 1
  3s - Active playbacks: 1
[Song: playback-1] Sample Look ahead 11.0 - 12.0
  4s - Active playbacks: 1
  5s - Active playbacks: 1
[Song: playback-1] Sample Look ahead 12.0 - 13.0
  6s - Active playbacks: 1
  7s - Active playbacks: 1
[Song: playback-1] Sample 